# crawling FM from the links and create full DF

## https://fminside.net/

We will take all the details of the players required for us from the website indicated above

In [21]:
import time      
import os        
import bs4 
from bs4 import BeautifulSoup  
import pandas as pd
import scipy as sc
import numpy as np
import requests

We throw duplicate lines in order to prevent players who received the same link, or we don't find a link for them at all, and this can interfere with the correctness of the model later on

In [22]:
df = pd.read_csv("players_link.csv")
df_2=df.copy()
df_2=df.drop_duplicates(subset=['link'],keep=False)
df_2=df_2.reset_index(drop=True)
df_2.drop(columns='column_1',inplace=True)
df_2


,value,name,club,nation,position,age,link
0,€180.00m,Kylian Mbappé,Paris Saint-Germain,France,Centre-Forward,24,https://fminside.net/players/3-fm-23/85139014-...
1,€170.00m,Erling Haaland,Manchester City,Norway,Centre-Forward,22,https://fminside.net/players/3-fm-23/29179241-...
2,€120.00m,Vinicius Junior,Real Madrid,Brazil,Left Winger,22,https://fminside.net/players/3-fm-23/19302146-...
3,€90.00m,Harry Kane,Tottenham Hotspur,England,Centre-Forward,29,https://fminside.net/players/3-fm-23/28049320-...
4,€85.00m,Rafael Leão,AC Milan,Portugal,Left Winger,23,https://fminside.net/players/3-fm-23/83111483-...
...,...,...,...,...,...,...,...
1377,€5.00m,Federico Gatti,Juventus FC,Italy,Centre-Back,24,https://fminside.net/players/3-fm-23/43491384-...
1378,€5.00m,Adrià Pedrosa,RCD Espanyol Barcelona,Spain,Left-Back,24,https://fminside.net/players/3-fm-23/95035991-...
1379,€5.00m,Joaquín Piquerez,Sociedade Esportiva Palmeiras,Uruguay,Left-Back,24,https://fminside.net/players/3-fm-23/78077014-...
1380,€5.00m,Nicolas Cozza,Montpellier HSC,France,Left-Back,23,https://fminside.net/players/3-fm-23/49037761-...


In [23]:
links=df_2['link']


# crawling

In [24]:
def get_tables(link):
    try: 
        html=requests.get(link)
        soup = BeautifulSoup(html.content, 'html.parser')
        tbl=soup.find("div",attrs={'class':'block stats'})
        rating=soup.find("span",attrs={'id':'ability'}).string
        tbl_2=soup("div",attrs={'class':'column'})[0]
        return tbl,tbl_2,rating
    except:
        return np.nan,np.nan,np.nan



With the help of the links we obtained thanks to Selenium
We will use crawling to extract data about each player

In [25]:
dict_Attributes={'Corners':[],'Crossing':[],'Dribbling':[],'Finishing':[],'First Touch':[],'Free Kick Taking':[],
       'Heading':[],'Long Shots':[],'Long Throws':[],'Marking':[],'Passing':[],'Penalty Taking':[],
       'Tackling':[],'Technique':[],'Aggression':[],'Anticipation':[],'Bravery':[],'Composure':[],
       'Concentration':[],'Decisions':[],'Determination':[],'Flair':[],'Leadership':[],'Off the Ball':[],
       'Positioning':[],'Teamwork':[],'Vision':[],'Work Rate':[],'Acceleration':[],'Agility':[],
       'Balance':[],'Jumping Reach':[],'Natural Fitness':[],'Pace':[],'Stamina':[],'Strength':[],'rating':[],'Foot':[]
       ,'Length':[],'Weight':[]}

list_keys = []
for key in dict_Attributes.keys():
    list_keys.append(key)

In [26]:
for link in links:
    table,table_2,rating=get_tables(link)
    if table==np.nan:
        for i in range(40):
            dict_Attributes[list_keys[i]].append(np.nan)
    else:
        for i in range(36):
              dict_Attributes[list_keys[i]].append(table('td',attrs={'class':'stat'})[i].string)
        dict_Attributes['rating'].append(rating)
        dict_Attributes['Foot'].append(table_2('span',attrs={'class':'value'})[3].string)
        dict_Attributes['Length'].append(table_2('span',attrs={'class':'value'})[4].string.split()[0])
        dict_Attributes['Weight'].append(table_2('span',attrs={'class':'value'})[5].string.split()[0])

In [29]:
df2=pd.DataFrame(dict_Attributes)
df2

,Corners,Crossing,Dribbling,Finishing,First Touch,Free Kick Taking,Heading,Long Shots,Long Throws,Marking,...,Balance,Jumping Reach,Natural Fitness,Pace,Stamina,Strength,rating,Foot,Length,Weight
0,13,13,18,17,18,12,7,13,4,4,...,15,8,15,20,14,11,94,Right,178,73
1,7,10,14,18,16,13,15,13,5,6,...,17,18,19,19,14,17,93,Left,195,88
2,7,13,18,16,13,8,11,12,4,3,...,14,8,15,17,14,12,87,Right,177,62
3,9,15,14,19,15,10,15,15,6,10,...,14,13,16,14,16,15,92,Both,188,86
4,5,15,17,15,16,6,11,13,6,4,...,16,14,15,16,11,14,82,Right,188,81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1377,7,7,8,10,10,7,15,7,8,13,...,12,15,14,13,14,15,65,Right,190,84
1378,7,9,14,6,11,2,12,11,8,14,...,13,8,10,15,13,10,66,Left,176,69
1379,11,12,12,10,13,12,12,12,12,14,...,14,12,13,13,13,14,66,Left,183,81
1380,9,11,11,8,13,7,12,9,10,14,...,14,12,13,12,14,13,63,Left,178,72


We will concat the dataframes

In [30]:
df_all=pd.concat([df_2,df2],axis=1)
df_all

,value,name,club,nation,position,age,link,Corners,Crossing,Dribbling,...,Balance,Jumping Reach,Natural Fitness,Pace,Stamina,Strength,rating,Foot,Length,Weight
0,€180.00m,Kylian Mbappé,Paris Saint-Germain,France,Centre-Forward,24,https://fminside.net/players/3-fm-23/85139014-...,13,13,18,...,15,8,15,20,14,11,94,Right,178,73
1,€170.00m,Erling Haaland,Manchester City,Norway,Centre-Forward,22,https://fminside.net/players/3-fm-23/29179241-...,7,10,14,...,17,18,19,19,14,17,93,Left,195,88
2,€120.00m,Vinicius Junior,Real Madrid,Brazil,Left Winger,22,https://fminside.net/players/3-fm-23/19302146-...,7,13,18,...,14,8,15,17,14,12,87,Right,177,62
3,€90.00m,Harry Kane,Tottenham Hotspur,England,Centre-Forward,29,https://fminside.net/players/3-fm-23/28049320-...,9,15,14,...,14,13,16,14,16,15,92,Both,188,86
4,€85.00m,Rafael Leão,AC Milan,Portugal,Left Winger,23,https://fminside.net/players/3-fm-23/83111483-...,5,15,17,...,16,14,15,16,11,14,82,Right,188,81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1377,€5.00m,Federico Gatti,Juventus FC,Italy,Centre-Back,24,https://fminside.net/players/3-fm-23/43491384-...,7,7,8,...,12,15,14,13,14,15,65,Right,190,84
1378,€5.00m,Adrià Pedrosa,RCD Espanyol Barcelona,Spain,Left-Back,24,https://fminside.net/players/3-fm-23/95035991-...,7,9,14,...,13,8,10,15,13,10,66,Left,176,69
1379,€5.00m,Joaquín Piquerez,Sociedade Esportiva Palmeiras,Uruguay,Left-Back,24,https://fminside.net/players/3-fm-23/78077014-...,11,12,12,...,14,12,13,13,13,14,66,Left,183,81
1380,€5.00m,Nicolas Cozza,Montpellier HSC,France,Left-Back,23,https://fminside.net/players/3-fm-23/49037761-...,9,11,11,...,14,12,13,12,14,13,63,Left,178,72


In [31]:
df_all.to_csv("players_Attributes")